# 批标准化 BatchNormalization

batchNormalization 批标准化，与普通的数据标准化相似，是将分散的数据统一的一种做法，也是优化神经网络的一种方法，因为具有统一规格的数据能够让机器学习更容易学习到其中的规律

## BN解决了ICS的问题吗？
ICS = Internal Covariate Shift
解决ICS的最基本办法就是对网络的输入作归一化，使得输入分布的均值为0，标准差为1，这个方法仅仅在网络不深的时候才奏效，网络一旦变深，每层参数引起的分布的微小变化叠加起来的变化是巨大的，所以我们需要对每一层的输入都做一些归一化操作。

从整体上来看，BN层的作用就是通过参数控制了每一层输出的均值和方差。

ICS问题是训练过程中，网络中间层输入分布的变化，BN层并没有解决ICS问题，而是引入了参数gamma和beta去调节中间层输出的均值和标准差，gamma和beta会在训练过程中不断更新，意味着均值和标准差也不断在变化，即BN本质上暗含了ICS。

实际上，BN能够提高收敛速度

## 使用BN层需要注意的细节
1. BN层是放在激活层之后的
2. 推理过程中没有batch情况下的主流解决方法：在训练时跟踪记录每一个batch的mean和var，然后使用这些值对全部样本的均值和标准差作无偏估计。Pytorch中model.eval()就是如此了
3. BN层的正则化作用：在BN层中每个batch计算得到的均值和方差都是对于全局的一个近似估计，这为我们最优解的搜索引入了随机性，从而起到了正则化的作用
4. BN的缺陷：带有BN层的网络错误率会随着batch_size的减小而迅速增大，当我们硬件条件受到限制不得不使用较小的batch_size的时候，网络的效果会大打折扣，后面LN、IN和GroupNormalization aka GN 是为了解决该问题的



在神经网络中，数据分布会对训练产生影响


In [17]:
# 有时候激励函数对于变化范围较大的数据不敏感
import torch as t
x1 = t.tensor(5,dtype=t.float32)
x2 = t.tensor(40,dtype=t.float32)

print(t.tanh(x1),t.tanh(x2))

tensor(0.9999) tensor(1.)


可以看到，tanh(5)约等于tanh(40),这是很糟糕的，所以要在层与层之间把数据处理一下，让他们都在激活函数的敏感区间之内。

训练深层神经网络十分困难，特别是我们的目标是要在较短时间内使它收敛的时候，BN是一种有效的技术，可持续加速深层网络的收敛速度。

从形式上来说，BN根据以下表达式转换输入X

$$\mathrm{BN}(\mathbf{x}) = \boldsymbol{\gamma} \odot \frac{\mathbf{x} - \hat{\boldsymbol{\mu}}_\mathcal{B}}{\hat{\boldsymbol{\sigma}}_\mathcal{B}} + \boldsymbol{\beta}.$$

其中拉伸参数gamma和偏移参数beta他们的形状与输入X相同，但是是属于nn.parameters的，需要被学习

## ***注意：在有BN层的网络中，选择合适的batch size比没有BN层的网络更加重要***

# 从头开始实现BN层

In [18]:
import torch as t
import torch.nn as nn
from torch import Tensor
def batch_norm(X:Tensor,
                gamma:Tensor,
                beta:Tensor,
                moving_mean:Tensor,
                moving_var:Tensor,
                eps:float,
                momentum:float)->tuple[Tensor,Tensor,Tensor]:
    # 通过is_grad_enabled 来判断当前模式是训练模式还是预测模式
    if not t.is_grad_enabled():
        # 如果是在预测模式下，直接使用传入的移动平均所得的均值和方差
        X_hat =(X-moving_mean)/t.sqrt(moving_var+eps)
    else:
        assert len(X.shape) in (2,4)
        
        # 如果使用全连接层，计算特征维上的均值和方差
        if len(X.shape)==2:
            mean = X.mean(dim=0)
            var = ((X-mean)**2).mean(dim = 0)
        # 如果使用卷积层，计算通道维上的均值和方差,即所有的某通道的数值和除以（行乘以列乘以batch数）
        else:
            mean = X.mean(dim=(0,2,3),keepdim=True)
            var = ((X-mean)**2).mean(dim = (0,2,3),keepdim=True)
        
        # 训练模式下，用当前的均值和方差做标准化
        X_hat =(X-mean)/t.sqrt(var+eps)
        # 更新移动平均的均值和方差
        moving_mean = momentum*moving_mean +(1-momentum)*mean
        moving_var = momentum*moving_var +(1-momentum)*var

    # 缩放和移位
    Y=gamma*X_hat+beta
    return Y,moving_mean.data,moving_var.data
    

# 自定义BN模块

In [19]:
class BatchNorm(nn.Module):
    def __init__(self,num_features:int,num_dims:int)->None:
        super().__init__()

        if num_dims==2:
            shape =(1,num_features)
        else:
            shape = (1,num_features,1,1)
        # 参与求梯度和迭代的拉伸和偏移参数，分别初始化成1和0
        self.gamma = nn.Parameter(t.ones(shape))
        self.beta =nn.Parameter(t.zeros(shape))
        # 非模型参数的变量初始化为0和1
        self.moving_mean = t.zeros(shape)
        self.moving_var = t.ones(shape )

    def forward(self,X:Tensor)->Tensor:
        if self.moving_mean.device!=X.device:
            self.moving_mean=self.moving_mean.to(X.device)
            self.moving_var=self.moving_var.to(X.device)
        #保存更新之后的moving_mean和moving_var
        Y,self.moving_mean,self.moving_var=batch_norm(X,self.gamma,self.beta,self.moving_mean,self.moving_var,eps=1e-5,momentum=0.9)
        return Y

批量规范化被认为可以使优化更加的平滑，但是有时候会表现出论文中相反的结果

批量规范化已经被证明是一种不可或缺的方法，它适用于几乎所有的图像分类器，并在学术界获得了数万引用。

# 关于 torch.Tensor.mean((0,2,3))的详细解析

In [20]:
# numerical calculations

import torch as t

X = t.arange(0,27,1,dtype=t.float32).reshape(1,3,3,3)
X.shape

torch.Size([1, 3, 3, 3])

In [21]:
X

tensor([[[[ 0.,  1.,  2.],
          [ 3.,  4.,  5.],
          [ 6.,  7.,  8.]],

         [[ 9., 10., 11.],
          [12., 13., 14.],
          [15., 16., 17.]],

         [[18., 19., 20.],
          [21., 22., 23.],
          [24., 25., 26.]]]])

In [22]:
Y= X.mean(0)
Y

tensor([[[ 0.,  1.,  2.],
         [ 3.,  4.,  5.],
         [ 6.,  7.,  8.]],

        [[ 9., 10., 11.],
         [12., 13., 14.],
         [15., 16., 17.]],

        [[18., 19., 20.],
         [21., 22., 23.],
         [24., 25., 26.]]])

In [23]:
Z = Y.mean(1)
Z

tensor([[ 3.,  4.,  5.],
        [12., 13., 14.],
        [21., 22., 23.]])

In [24]:
O = Z.mean(1)
O

tensor([ 4., 13., 22.])

In [25]:
# equals this
X.mean((0,2,3))

tensor([ 4., 13., 22.])

# batch normalization 的Pytorch 实现

In [26]:
import torch as t
import torch.nn as nn
from torch import Tensor

bn = nn.BatchNorm2d(3)
X = t.arange(0, 27, 1, dtype=t.float32).reshape(1, 3, 3, 3)
X.shape


torch.Size([1, 3, 3, 3])

In [27]:
for name,param in bn.named_parameters():
    print(name, param)
# 三个通道，所以gamma 和 beta 均有三个参数

weight Parameter containing:
tensor([1., 1., 1.], requires_grad=True)
bias Parameter containing:
tensor([0., 0., 0.], requires_grad=True)


In [28]:
bn.forward(X)

tensor([[[[-1.5492, -1.1619, -0.7746],
          [-0.3873,  0.0000,  0.3873],
          [ 0.7746,  1.1619,  1.5492]],

         [[-1.5492, -1.1619, -0.7746],
          [-0.3873,  0.0000,  0.3873],
          [ 0.7746,  1.1619,  1.5492]],

         [[-1.5492, -1.1619, -0.7746],
          [-0.3873,  0.0000,  0.3873],
          [ 0.7746,  1.1619,  1.5492]]]], grad_fn=<NativeBatchNormBackward0>)

In [29]:
# 来看看我们自己的bn是不是结果一样？
bn2 = BatchNorm(3,4)
bn2.forward(X)
# 雀氏是一样的↓

tensor([[[[-1.5492, -1.1619, -0.7746],
          [-0.3873,  0.0000,  0.3873],
          [ 0.7746,  1.1619,  1.5492]],

         [[-1.5492, -1.1619, -0.7746],
          [-0.3873,  0.0000,  0.3873],
          [ 0.7746,  1.1619,  1.5492]],

         [[-1.5492, -1.1619, -0.7746],
          [-0.3873,  0.0000,  0.3873],
          [ 0.7746,  1.1619,  1.5492]]]], grad_fn=<AddBackward0>)

# 查看nn.BatchNorm2d的参数 running mean 和 running var

In [30]:
bn.running_mean



tensor([0.4000, 1.3000, 2.2000])

In [31]:
bn.running_var


tensor([1.6500, 1.6500, 1.6500])